In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import copy

% matplotlib inline

import sem
import story_generation
from event_schemas import LinearMLP, BasicRNN, LinearRNN, BoundedRNN
import hrr

### HRR Sanity Check

In [ ]:
dim = 5

In [ ]:
a = hrr.embed_2d(dim, None)
b = hrr.embed_2d(dim, None)
c = hrr.encode(a, b)
print a
print b
print c

In [ ]:
print hrr.normalize(a)
print hrr.normalize(hrr.decode(c, b))
print 
print hrr.normalize(b)
print hrr.normalize(hrr.decode(c, a))

### HRR Reconstruction

In [ ]:
# euclidean assignment to closest object
def eucl_match(prediction, objects):
    dists = list(map(lambda x : np.linalg.norm(prediction - x), objects))
    index = dists.index(min(dists)) 
    return objects[index]

def semantic_error(constants, predicted_scene, semantic_truth):
    decode_subj = hrr.decode(predicted_scene, constants['placeholders']['subject'])
    decode_subj_noun = hrr.decode(decode_subj, constants['placeholders']['noun'])
    decode_subj_property = hrr.decode(decode_subj, constants['placeholders']['property'])
    decode_verb = hrr.decode(predicted_scene, constants['placeholders']['verb'])
    decode_obj = hrr.decode(predicted_scene, constants['placeholders']['object'])
    decode_obj_noun = hrr.decode(decode_obj, constants['placeholders']['noun'])
    decode_obj_property = hrr.decode(decode_obj, constants['placeholders']['property'])
    
    normalized_nouns = list(map(lambda x: hrr.normalize(x), constants['nouns'].values())) + list(map(lambda x: hrr.normalize(x['vector']), actors))
    normalized_verbs = list(map(lambda x: hrr.normalize(x), constants['verbs'].values()))
    normalized_properties = list(map(lambda x: hrr.normalize(x), constants['properties'].values()))
    
    pred_subj_noun = eucl_match(hrr.normalize(decode_subj_noun), normalized_nouns)
    pred_subj_property = eucl_match(hrr.normalize(decode_subj_property), normalized_properties)
    pred_verb = eucl_match(hrr.normalize(decode_verb), normalized_verbs)
    pred_obj_noun = eucl_match(hrr.normalize(decode_obj_noun), normalized_nouns)
    pred_obj_property = eucl_match(hrr.normalize(decode_obj_property), normalized_properties)

    matched = 0
    if list(pred_subj_noun) != list(hrr.normalize(semantic_truth['subject_noun'])):
        matched += 1
    if list(pred_subj_property) != list(hrr.normalize(semantic_truth['subject_property'])):
        matched += 1
    if list(pred_verb) != list(hrr.normalize(semantic_truth['verb'])):
        matched += 1
    if list(pred_obj_noun) != list(hrr.normalize(semantic_truth['object_noun'])):
        matched += 1
    if list(pred_obj_property) != list(hrr.normalize(semantic_truth['object_property'])):
        matched += 1
    return matched/5.0

In [ ]:
actors, constants = story_generation.initialize_vectors(dim)
training, semantic_vectors = story_generation.generate_evaluation_events(actors, constants, 100)

In [ ]:
demo = LinearRNN(dim)
# training is a set of sequences
semantic_errors = []
euclidean_errors = []
preds = []
for e in range(len(training)):
    event = training[e]
    seen = [event[0]]
    sem_error = 0
    eucl_error = 0
    count = 0
    pred = []
    for s in range(1, len(event)):
        scene = event[s]
        count += 1
        prediction = demo.predict(seen)
        sem_error += semantic_error(constants, prediction, semantic_vectors[e][s])
        eucl_error += np.linalg.norm(prediction - scene)
        seen.append(scene)
        pred.append(prediction)
    semantic_errors.append(sem_error / count)
    euclidean_errors.append(eucl_error / count)
    demo.train_recurrent(event)
    preds.append(pred)

In [ ]:
plt.plot(range(len(euclidean_errors)), euclidean_errors)

In [ ]:
plt.plot(range(len(semantic_errors)), semantic_errors)